In [9]:
import numpy as np
import pymc as pm
import arviz as az

from scipy.stats import gamma
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

EX1.a

In [10]:
data = np.array([56, 60, 58, 55, 57, 59, 61, 56, 58, 60])

prior_mu = np.mean(data)

with pm.Model() as model:
    mu = pm.Normal("mu", mu=prior_mu, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=10)
    X_obs = pm.Normal("X_obs", mu=mu, sigma=sigma, observed=data)

EX1.b

In [11]:
with model:
    step = pm.Metropolis()
    trace = pm.sample(5000, step=step, tune=1000, return_inferencedata=True, random_seed=42)

hdi_mu = az.hdi(trace, hdi_prob=0.95)["mu"].values
hdi_sigma = az.hdi(trace, hdi_prob=0.95)["sigma"].values

print("b) Bayesian estimates with 95% HDI:")
print("95% HDI for μ:", hdi_mu)
print("95% HDI for σ:", hdi_sigma)

Output()

b) Bayesian estimates with 95% HDI:
95% HDI for μ: [56.38069729 59.54200318]
95% HDI for σ: [1.28279477 3.64926858]


Ex1.c

In [12]:
sample_mean = np.mean(data)
sample_std = np.std(data, ddof=1)

print("Sample mean (mu):", sample_mean)
print("Sample std (sigma):", sample_std)

posterior_mean_mu = trace.posterior['mu'].mean().values
posterior_mean_sigma = trace.posterior['sigma'].mean().values

print("c) Bayesian posterior means:")
print("Posterior mean mu:", posterior_mean_mu)
print("Posterior mean sigma:", posterior_mean_sigma)
print("""
- Estimarea frecventiala a lui mu este media esantionului (≈ 58.0), care este foarte apropiata de media a posteriori bayesiana.
- Media a posteriori bayesiana a lui sigma poate diferi usor de deviatia standard a esantionului datorita priorului HalfNormal, care previne ca sigma sa fie nerealist de mic.
- Diferentele apar deoarece inferenta bayesiana ia in calcul informatia a priori si incertitudinea completa a posteriorului, in timp ce estimarile frecventiale se bazeaza doar pe datele observate.
- Cu un prior slab informativ si date suficiente, estimarile bayesiene si frecventiale sunt de obicei foarte apropiate.
""")

Sample mean (mu): 58.0
Sample std (sigma): 2.0
c) Bayesian posterior means:
Posterior mean mu: 57.99935017314219
Posterior mean sigma: 2.3420954417280724

- Estimarea frecventiala a lui mu este media esantionului (≈ 58.0), care este foarte apropiata de media a posteriori bayesiana.
- Media a posteriori bayesiana a lui sigma poate diferi usor de deviatia standard a esantionului datorita priorului HalfNormal, care previne ca sigma sa fie nerealist de mic.
- Diferentele apar deoarece inferenta bayesiana ia in calcul informatia a priori si incertitudinea completa a posteriorului, in timp ce estimarile frecventiale se bazeaza doar pe datele observate.
- Cu un prior slab informativ si date suficiente, estimarile bayesiene si frecventiale sunt de obicei foarte apropiate.



EX1.d

In [13]:
with pm.Model() as strong_prior_model:
    mu = pm.Normal("mu", mu=50, sigma=1)
    sigma = pm.HalfNormal("sigma", sigma=10)
    X_obs = pm.Normal("X_obs", mu=mu, sigma=sigma, observed=data)
    step = pm.Metropolis()
    strong_trace = pm.sample(5000, step=step, tune=1000, return_inferencedata=True, random_seed=42)

posterior_strong_mu = strong_trace.posterior['mu'].mean().values
posterior_strong_sigma = strong_trace.posterior['sigma'].mean().values

hdi_strong_mu = az.hdi(strong_trace, hdi_prob=0.95)["mu"].values
hdi_strong_sigma = az.hdi(strong_trace, hdi_prob=0.95)["sigma"].values

print("Part d) Bayesian estimates with strong prior:")
print("Posterior mean mu:", posterior_strong_mu)
print("95% HDI μ:", hdi_strong_mu)
print("Posterior mean sigma:", posterior_strong_sigma)
print("95% HDI sigma:", hdi_strong_sigma)

print("""
Explicatie:

- Folosind un prior puternic pentru mu (N(50,1)), posteriorul pentru mu este tras catre 50, chiar daca datele sugereaza o valoare mai mare (~58).
- Efectul acestui prior puternic este ca datele nu mai domina complet posteriorul, iar estimarea bayesiana pentru mu se apropie mult de media priorului.
- Estimarea pentru sigma poate ramane aproape de cea anterioara deoarece priorul pentru sigma nu s-a schimbat.
- Diferentele apar din cauza influentei mari a priorului puternic: in Bayesian inference, priorul si datele se combina pentru a forma posteriorul, deci un prior foarte informativ poate domina rezultatele daca datele sunt putine.
""")



Output()

Part d) Bayesian estimates with strong prior:
Posterior mean mu: 51.35759433781182
95% HDI μ: [49.27821964 53.51314451]
Posterior mean sigma: 7.699083848309568
95% HDI sigma: [ 3.88685516 12.13744419]

Explicatie:

- Folosind un prior puternic pentru mu (N(50,1)), posteriorul pentru mu este tras catre 50, chiar daca datele sugereaza o valoare mai mare (~58). 
- Efectul acestui prior puternic este ca datele nu mai domina complet posteriorul, iar estimarea bayesiana pentru mu se apropie mult de media priorului.
- Estimarea pentru sigma poate ramane aproape de cea anterioara deoarece priorul pentru sigma nu s-a schimbat.
- Diferentele apar din cauza influentei mari a priorului puternic: in Bayesian inference, priorul si datele se combina pentru a forma posteriorul, deci un prior foarte informativ poate domina rezultatele daca datele sunt putine.

